# Initial Analaysis of outputs

In [1]:
from date_parser import to_date
import os
import json
import pandas as pd
from collections import defaultdict
import ast
from tqdm import tqdm
import numpy as np
from datetime import datetime, timedelta, timezone


solution_path = "fctryengine_outputs/schedule.json"
datapath = 'fctryengine_outputs/data/'


def load_json_from_path(path):
    with open(path,'r') as file:
        return json.load(file)


def convert_minutes(minutes):
    days = minutes // (24 * 60)  # Total days
    hours = (minutes % (24 * 60)) // 60  # Remaining hours
    minutes = minutes % 60  # Remaining minutes

    return days, hours, minutes



def convert_string_to_list(input_string):
    # Extracting the inner list representation from the string
    inner_list_str = input_string.strip('dict_values()')
    
    # Safely evaluating the string as a Python literal
    inner_list = ast.literal_eval(inner_list_str)

    # Flatten the list of lists if needed and return
    return [item for sublist in inner_list for item in sublist][0]


def union_of_lists(list_of_lists):
    result = []
    for sublist in list_of_lists:
        result +=sublist
    return set(result)


def minutes_since(date_str, reference_date_str = '2023-11-27 00:00:00'):
    """
    Calculates how many minutes have passed since a reference date.

    Parameters:
    date_str (str): A date in the format 'YYYY-MM-DD HH:MM:SS+00:00'.
    reference_date_str (str): A reference date in the format 'DD-MM-YYYY'.

    Returns:
    int: Number of minutes that have passed since the reference date.
    """

    date_str = date_str.split('+')[0]

    # Convert the date strings to datetime objects
    date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    reference_date = datetime.strptime(reference_date_str, '%Y-%m-%d %H:%M:%S')
    

    # Calculate the difference in minutes
    difference = date - reference_date
    return int(difference.total_seconds() // 60)


def add_minutes_to_date(minutes, reference_date_str = '2023-11-27 00:00:00'):
    """
    Adds a certain number of minutes to a reference date.

    Parameters:
    minutes (int): The number of minutes to add.
    reference_date_str (str): A reference date in the format 'YYYY-MM-DD HH:MM:SS'.

    Returns:
    datetime: The new date and time.
    """
    # Convert the reference date string to a datetime object
    reference_date = datetime.strptime(reference_date_str, '%Y-%m-%d %H:%M:%S')

    # Add the minutes to the reference date
    new_date = reference_date + timedelta(minutes=minutes)

    return new_date


def convert_date_to_datetime(date_str):
    """
    Converts a date string in the format 'YYYY-MM-DD' to a datetime object with time set to 00:00:00.

    Parameters:
    date_str (str): A date in the format 'YYYY-MM-DD'.

    Returns:
    datetime: The datetime object.
    """
    date_str = date_str + " 00:00:00"
    date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    return date




all_data_files = {f.split('.')[0]:load_json_from_path(datapath+f) for f in  os.listdir(datapath)}
solution = load_json_from_path(solution_path)


In [2]:
# PLAN
all_data_files['tasks']

[{'taskno': 'WO20946-10',
  'task_id': 79828,
  'priority': 323,
  'duration': 30,
  'quantity': 1.0,
  'predecessors': [],
  'total_resource': 2,
  'micro_batch_size': None,
  'flow_first_step_id': None,
  'parent_item_collection_id': None,
  'predecessor_item_collection_id': None,
  'use_all_resources': False,
  'resource_group_id': [38],
  'resource_count': 1,
  'rule_id': 24},
 {'taskno': 'WO29559-10',
  'task_id': 102804,
  'priority': 328,
  'duration': 30,
  'quantity': 1.0,
  'predecessors': [],
  'total_resource': 7,
  'micro_batch_size': None,
  'flow_first_step_id': None,
  'parent_item_collection_id': None,
  'predecessor_item_collection_id': None,
  'use_all_resources': False,
  'resource_group_id': [67],
  'resource_count': 1,
  'rule_id': 7},
 {'taskno': 'WO30798-20',
  'task_id': 106244,
  'priority': 324,
  'duration': 120,
  'quantity': 1.0,
  'predecessors': [],
  'total_resource': 1,
  'micro_batch_size': None,
  'flow_first_step_id': None,
  'parent_item_collection

In [3]:

task2job = {}
task2RG = {}
task2duration = {}

task_dict = defaultdict(list)
for task in all_data_files['tasks']:
    task_dict['priority_id'].append(task['priority'])
    task_dict['taskno'].append(task['taskno'])
    task_dict['task_id'].append(task['task_id'])
    task_dict['resource_group_id'].append(task['resource_group_id'])
    task_dict['total_resource'].append(task['total_resource'])
    task_dict['duration'].append(task['duration'])
    task_dict['predecessors'].append(task['predecessors'])
    task_dict['quantity'].append(task['quantity'])

    task2duration[task['taskno']] = task['duration']


    task2job[task['taskno']] = task['priority']

    task2RG[task['taskno']] = task['resource_group_id']

task_frame = pd.DataFrame(task_dict)
task_frame

,priority_id,taskno,task_id,resource_group_id,total_resource,duration,predecessors,quantity
0,323,WO20946-10,79828,[38],2,30,[],1.0
1,328,WO29559-10,102804,[67],7,30,[],1.0
2,324,WO30798-20,106244,[3],1,120,[],1.0
3,339,WO30819-20,106255,[3],1,30,[],1.0
4,325,WO30820-20,106257,[3],1,30,[],2.0
...,...,...,...,...,...,...,...,...
3173,303,WO117905-10,396966,[36],1,30,[],2.0
3174,303,WO117906-10,396967,[36],1,30,[],2.0
3175,302,WO117907-10,396968,[36],1,30,[],4.0
3176,303,WO117908-10,396969,[84],4,30,[],4.0


In [4]:
task

{'taskno': 'WO117909-10',
 'task_id': 396970,
 'priority': 291,
 'duration': 2366,
 'quantity': 1.0,
 'predecessors': ['WO103020-30'],
 'total_resource': 7,
 'micro_batch_size': None,
 'flow_first_step_id': None,
 'parent_item_collection_id': None,
 'predecessor_item_collection_id': None,
 'use_all_resources': False,
 'resource_group_id': [67],
 'resource_count': 1,
 'rule_id': 7}

In [5]:
task_frame.loc[task_frame['taskno'].str.contains('WO111842-10')]

,priority_id,taskno,task_id,resource_group_id,total_resource,duration,predecessors,quantity
331,237,WO111842-10,368244,[64],5,30,[],36.0


In [6]:
res2RG  = {}
for rg in all_data_files['groups']:
    for rid in rg['resource_id']:
        res2RG[rid] = rg['resource_group_id']

In [7]:
task_frame.loc[task_frame['predecessors'].str.len()>2]

,priority_id,taskno,task_id,resource_group_id,total_resource,duration,predecessors,quantity
63,173,WO104119-10,333290,"[105, 97]",4,5475,"[WO115273-10, WO104121-20, WO104571-10]",1.0
64,175,WO104120-10,333293,"[105, 97]",4,10106,"[WO104436-10, WO104442-10, WO104437-10, WO1044...",1.0
114,29,WO107571-10,347265,[67],7,22095,"[WO107644-10, WO107645-10, WO107643-10, WO1076...",1.0
115,30,WO107572-10,347266,[67],7,22785,"[WO107630-10, WO107627-10, WO107632-10, WO1076...",1.0
121,30,WO107633-10,347468,[38],2,30,"[WO114308-30, WO108125-10, WO114304-30, WO1143...",1.0
...,...,...,...,...,...,...,...,...
3057,304,WO117823-10,395943,"[16, 17]",9,1400,"[WO117870-20, WO117887-10, WO117883-10, WO1178...",1.0
3059,301,WO117824-10,395945,"[16, 17]",9,1600,"[WO117869-20, WO117881-10, WO117868-20, WO1178...",1.0
3061,302,WO117825-10,395947,"[16, 17]",9,1500,"[WO117891-20, WO117880-10, WO117865-20, WO1178...",2.0
3063,28,WO117826-10,395949,"[2, 1]",9,240,"[WO117829-10, WO117828-10, WO117830-10]",1.0


In [8]:
solution_dict = defaultdict(list)

count = 0


failed_uids = defaultdict(int)


handled_tasks = defaultdict(bool)

for schedule in tqdm(solution):
    try:
        task2job[schedule['task_uid']]
    except:
        failed_uids['-'.join(schedule['task_uid'].split('-')[:2])]+=1
        count+=1
    
    RGs = list(set([res2RG[int(rid)] for rid in schedule["assigned_resource_ids"]]))

    taskuid = '-'.join(schedule['task_uid'].split('-')[:2])
    if handled_tasks[taskuid]:
        continue
    else:
        handled_tasks[taskuid]=True

    
    for RG in RGs:
        

        solution_dict['JobID'].append(task2job[taskuid])
        solution_dict['TaskID'].append(taskuid)
        solution_dict['ResourceGroup'].append(RG)

        solution_dict['duration'].append(task2duration[taskuid])
        start_,end_ = convert_string_to_list(schedule['resource_intervals'])
        solution_dict['start_time'].append(start_)
        days, hours, minutes = convert_minutes(start_)
        solution_dict['days'].append(days)
        solution_dict['hours'].append(hours)
        solution_dict['minutes'].append(minutes)

solution_frame = pd.DataFrame(solution_dict)

#failed_uids

  0%|          | 0/3704 [00:00<?, ?it/s]

100%|██████████| 3704/3704 [00:00<00:00, 19674.71it/s]


# EDA

In [9]:
solution_frame['JobID'].unique().shape[0]

338

In [10]:
solution_frame['TaskID'].unique().shape[0]

3178

In [11]:
d,h,m = convert_minutes(solution_frame['duration'].sum())
print(f"{d} Days {h} Hours and {m} Minutes of Work")

last_job = solution_frame.loc[solution_frame["start_time"]==solution_frame["start_time"].max()]
d,h,m = convert_minutes(last_job['start_time'].values[0]+last_job['duration'].values[0])

print(f"Final Job finished in {d} Days {h} Hours and {m} Minutes") 

1280 Days 18 Hours and 27 Minutes of Work
Final Job finished in 197.0 Days 21.0 Hours and 50.0 Minutes


In [12]:
def getRGsets(all_data_files):
    # Create workday lookup
    resource2wd = {resource['resource_id']:resource['availability'] for resource in all_data_files['resource']}

    RG_Sets = defaultdict(list)

    for RG in tqdm(all_data_files['groups']):
        for resource_id in RG['resource_id']:
            try:
                wds = resource2wd[resource_id]
            except:
                print('MISSING:',resource_id)
                continue
            for wd in wds:
                start_,end_ = minutes_since(wd['start_datetime']),minutes_since(wd['end_datetime'])
                if start_ > -1:
                    cur_window = [minute for minute in range(start_,end_+1)]
                    RG_Sets[RG['resource_group_id']].append(cur_window)

    print('AGGREGATING')
    RG_sets_aggregated = {RGid:union_of_lists(windows) for RGid,windows in tqdm(RG_Sets.items())}
    return RG_Sets,RG_sets_aggregated

RG_Sets,RG_sets_aggregated = getRGsets(all_data_files)



  1%|▏         | 1/74 [00:00<00:20,  3.58it/s]

MISSING: 44


 15%|█▍        | 11/74 [00:04<00:29,  2.12it/s]

MISSING: 109


 16%|█▌        | 12/74 [00:06<00:52,  1.17it/s]

...

 96%|█████████▌| 71/74 [00:21<00:00,  3.92it/s]

MISSING: 99


100%|██████████| 74/74 [00:21<00:00,  3.45it/s]


AGGREGATING


100%|██████████| 71/71 [00:05<00:00, 12.56it/s]


In [13]:
solution_frame.loc[solution_frame['JobID']==237]

,JobID,TaskID,ResourceGroup,duration,start_time,days,hours,minutes
306,237,WO111842-10,64,30,390.0,0.0,6.0,30.0
307,237,WO111842-40,65,30,390.0,0.0,6.0,30.0
308,237,WO111842-50,88,3330,900.0,0.0,15.0,0.0
309,237,WO111842-50,37,3330,900.0,0.0,15.0,0.0
417,237,WO112556-10,96,15000,115590.0,80.0,6.0,30.0
...,...,...,...,...,...,...,...,...
497,237,WO113074-20,91,188,20850.0,14.0,11.0,30.0
498,237,WO113082-10,103,55,4896.0,3.0,9.0,36.0
499,237,WO113082-20,98,30,32550.0,22.0,14.0,30.0
500,237,WO113085-10,98,30,12375.0,8.0,14.0,15.0


In [14]:
all_data_files['tasks'][0]

{'taskno': 'WO20946-10',
 'task_id': 79828,
 'priority': 323,
 'duration': 30,
 'quantity': 1.0,
 'predecessors': [],
 'total_resource': 2,
 'micro_batch_size': None,
 'flow_first_step_id': None,
 'parent_item_collection_id': None,
 'predecessor_item_collection_id': None,
 'use_all_resources': False,
 'resource_group_id': [38],
 'resource_count': 1,
 'rule_id': 24}

In [15]:
def get_predecessor_finish(WOids,WO2finish):
    return max([WO2finish[WOid] for WOid in WOids])



def get_overlap_between_RG_and_idle_times(idle_times,task2RG,RG_sets_aggregated):
    print('GETTING IDLE MINUTES')

    idle_minutes = {}

    idle_RGs = defaultdict(list)

    for WOid,idle_time in tqdm(idle_times.items()):
        RGs = task2RG[WOid]
        window = []
        for RG in RGs:
            window+=RG_sets_aggregated[RG]

        
        window = list(set(window))


        idle_range = np.intersect1d(window,idle_time).tolist()

        idle_minutes[WOid] = idle_range

        for RG in RGs:
            idle_RGs[RG]+=idle_range

    idle_RGs = {k:len(set(v)) for k,v in idle_RGs.items()}
    
    return idle_minutes,idle_RGs




def get_task_idle_times(solution,all_data_files,RG_sets_aggregated):

    # Task to Resource Group
    task2RG = {'-'.join(task['taskno'].split('-')) : task['resource_group_id'] for task in all_data_files['tasks']}

    WO2finish = {'-'.join(task['task_uid'].split('-')):task['task_end'] for task in solution}

    task_has_predecessors = {task['taskno']:task['predecessors'] for task in all_data_files['tasks']}

    idle_times  = {}

    # For each task
    print('CREATING DATA STRUCTURE')
    for task in tqdm(solution):
        task_uid = '-'.join(task['task_uid'].split('-')[:2])
        # Check if there are predecessors

        if len(task_has_predecessors[task_uid])>0:
             # If there are precursors find the maximum end time
             # assign the range from (max_precursor_end : start_time) as idle time
            try:
                idle_start = int(get_predecessor_finish(task_has_predecessors[task_uid],WO2finish))
                idle_end = int(task['task_start'])
            except:
                print('failed on predecessors of',task_uid)
                continue

        else:
            # if not assign the range from 0 to start_time as idle_time
            idle_start = 0
            idle_end = int(task['task_start'])


        idle_times[task_uid] = [minute for minute in range(idle_start,idle_end)]


    
    # Get RGs working during idle_times
    idle_minutes,idle_RGs = get_overlap_between_RG_and_idle_times(idle_times,task2RG,RG_sets_aggregated)
    
    # return dictionary {WOid : idle_range} 

    return idle_minutes,idle_RGs

In [16]:
idle_minutes,idle_RGs = get_task_idle_times(solution,all_data_files,RG_sets_aggregated)




CREATING DATA STRUCTURE


 13%|█▎        | 484/3704 [00:00<00:02, 1117.57it/s]

failed on predecessors of WO111393-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30
failed on predecessors of WO112016-30


 24%|██▍       | 905/3704 [00:00<00:02, 1309.81it/s]

failed on predecessors of WO114415-10
failed on predecessors of WO114415-10
failed on predecessors of WO114415-10
failed on predecessors of WO114415-10
failed on predecessors of WO114415-10
failed on predecessors of WO114415-10
failed on predecessors of WO114415-20
failed on predecessors of WO114415-20
failed on predecessors of WO114415-20
failed on predecessors of WO114415-20
failed on predecessors of WO114415-20
failed on predecessors of WO114415-20
failed on predecessors of WO114415-30
failed on predecessors of WO114415-30
failed on predecessors of WO114415-30
failed on predecessors of WO114415-30
failed on predecessors of WO114415-30
failed on predecessors of WO114415-30
failed on predecessors of WO114415-40
failed on predecessors of WO114415-40
failed on predecessors of WO114415-40
failed on predecessors of WO114415-40
failed on predecessors of WO114415-40
failed on predecessors of WO114415-40
failed on predecessors of WO114415-50
failed on predecessors of WO114415-50
failed on pr

...

 73%|███████▎  | 2692/3704 [00:02<00:01, 738.23it/s]

failed on predecessors of WO117167-10
failed on predecessors of WO117170-10
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-20
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on predecessors of WO117220-30
failed on pr

100%|██████████| 3704/3704 [00:05<00:00, 720.11it/s]


GETTING IDLE MINUTES


100%|██████████| 3125/3125 [04:43<00:00, 11.03it/s]


In [17]:
convert_minutes(len(list(set(idle_minutes['WO109978-10']))))

(1, 6, 12)

In [18]:
# ANTAL dage ALLE TASKS VENTER PÅ DENNE RESOURCE GROUP 
convert_minutes(idle_RGs[3])

(22, 10, 3)

In [19]:
# WO DF
# Read In Due dates

## WorkOrder DF
### Inspect Due Dates

In [20]:
duedates = pd.read_excel("fctryengine_outputs/EXW_Dates.xlsx",engine='openpyxl')

print(duedates.shape[0])
duedates.head()


1000


,taskno,exw_delivery_date
0,WO109979-10,2023-11-22
1,WO109978-10,2023-11-28
2,WO109804-10,2023-11-22
3,WO109805-10,2023-11-22
4,WO109812-10,2023-11-22


In [21]:
solution[0]

{'task_uid': 'WO100090-40',
 'assigned_resource_ids': ['114'],
 'task_start': 1925.0,
 'task_end': 1955.0,
 'resource_intervals': 'dict_values([[(1925.0, 1955.0)]])'}

In [22]:
# Columns
## Priority ID
## WOid
## Minutes idle
## Cur ETA
## DueDate
## Cur delay
## delay without idle-time



# Lookup Definitions
ETA_lookup = {'-'.join(s['task_uid'].split('-')[:2]) : add_minutes_to_date(s['task_end']) for s in solution}
duedate_lookup = {row['taskno']: convert_date_to_datetime(row['exw_delivery_date']) for k,row in duedates.iterrows()}


WorkOrder_dict = defaultdict(list)

for WOid, im in idle_minutes.items():

    WorkOrder_dict['Priority'].append(task2job[WOid])

    WorkOrder_dict['WorkOrder'].append(WOid)

    # Add resource groups 

    WorkOrder_dict['capable_resource_groups'].append('|'.join([str(i) for i in task2RG[WOid]]))

    
    WorkOrder_dict['idle_minutes'].append(len(im))


    # GET ETA FROM SOLUTION
    WorkOrder_dict['ETA'].append(ETA_lookup[WOid])

    # GET DUE DATES FROM duedates
    try:
        dd = duedate_lookup[WOid]
    except:
        dd = None
    WorkOrder_dict['DueDate'].append(dd)



for k,v in WorkOrder_dict.items():
    print(k,len(v))

WorkOrders = pd.DataFrame(WorkOrder_dict)


# Calculate dependant columns

def calculate_timedelta(df, col1, col2):
    df['Delay'] = df[col1] - df[col2]
    return df

WorkOrders = calculate_timedelta(WorkOrders, 'ETA', 'DueDate')
WorkOrders['DelayWithoutIdleTime'] = WorkOrders['Delay'] - pd.to_timedelta(WorkOrders['idle_minutes'], unit='m')
WorkOrders['TimeDifference'] = WorkOrders['Delay'] - WorkOrders['DelayWithoutIdleTime']

WorkOrders.loc[(~WorkOrders['DueDate'].isna())&(WorkOrders['DelayWithoutIdleTime']>pd.Timedelta(0))].sort_values('TimeDifference',ascending=False).head(30)


Priority 3125
WorkOrder 3125
capable_resource_groups 3125
idle_minutes 3125
ETA 3125
DueDate 3125


,Priority,WorkOrder,capable_resource_groups,idle_minutes,ETA,DueDate,Delay,DelayWithoutIdleTime,TimeDifference
689,281,WO114449-10,58,34406,2024-03-07 14:11:00.000,2024-01-31,36 days 14:11:00,12 days 16:45:00,23 days 21:26:00
690,281,WO114455-10,58,33038,2024-03-04 12:28:00.000,2024-01-31,33 days 12:28:00,10 days 13:50:00,22 days 22:38:00
339,276,WO112156-10,58,31460,2024-02-28 08:58:00.000,2024-01-23,36 days 08:58:00,14 days 12:38:00,21 days 20:20:00
341,276,WO112157-10,58,30618,2024-02-23 11:05:00.000,2024-01-23,31 days 11:05:00,10 days 04:47:00,21 days 06:18:00
346,275,WO112163-10,58,29040,2024-02-21 08:58:00.000,2024-01-23,29 days 08:58:00,9 days 04:58:00,20 days 04:00:00
343,275,WO112159-10,58,27672,2024-02-16 09:20:00.000,2024-01-23,24 days 09:20:00,5 days 04:08:00,19 days 05:12:00
433,237,WO112750-10,58,24700,2024-02-09 09:50:00.000,2023-12-20,51 days 09:50:00,34 days 06:10:00,17 days 03:40:00
435,237,WO112754-10,58,24200,2024-02-05 14:50:00.000,2023-12-20,47 days 14:50:00,30 days 19:30:00,16 days 19:20:00
525,236,WO114078-20,58,22306,2024-02-02 11:18:00.000,2023-12-20,44 days 11:18:00,28 days 23:32:00,15 days 11:46:00
552,236,WO114125-10,58,21464,2024-01-29 09:35:00.000,2023-12-20,40 days 09:35:00,25 days 11:51:00,14 days 21:44:00


In [23]:
WorkOrders.loc[~WorkOrders['DueDate'].isna()].sort_values('Priority').head(30)
WorkOrders[['TimeDifference','capable_resource_groups']].groupby('capable_resource_groups').sum().sort_values("TimeDifference",ascending=False)



,TimeDifference
capable_resource_groups,
58,377 days 12:19:00
9,341 days 12:05:00
84,271 days 02:53:00
38,196 days 04:22:00
69,120 days 02:29:00
67,118 days 22:31:00
31,101 days 20:00:00
2|1,89 days 23:12:00
16|17,39 days 13:57:00


In [24]:
from datetime import timedelta



## ResourceGroup DF

# Progress Report
1. Beregner IDLE TIME som intersektionen mellem en resource groups arbejdstid og tid før WO starter
2. Dette tager ikke højde for at IDLE-time overlapper, altså at der er flere opgaver der er IDLE samtidig. Dette skal nok fixes? (MÅSKE I get_overlap_between_RG_and_idle_times?)
3. Målet er at kunne sige hvor mange penge en potentiel flaskehals koster. Se skema i Notes